In [ ]:
import numpy as np
import pandas as pd
import os
import umap
import matplotlib.pyplot as plt
import seaborn as sb

# https://github.com/jingw2/size_constrained_clustering
#from size_constrained_clustering import equal

In [ ]:
# Load data
assay_file = '../assay_data/assay_matrix_discrete_270_assays.csv'

# Those feature and similarity files are available on Zenodo (dataset)
ge_file = '../feature_data/ge.npz'
ge_scaled_file = '../feature_data/ge_scaled.npz'
mo_file = '../feature_data/mo.npz'
cp_file = '../feature_data/cp.npz'
mobc_file = '../feature_data/mobc.npz'

similarity_fingerprints = '../misc/similarity_fingerprints.npz'

with open(ge_file, "rb") as data:
    ge_np = np.load(data)['features']
        
with open(mo_file, "rb") as data:
    mo_np = np.load(data)['features']
        
with open(ge_scaled_file, "rb") as data:
    ge_scaled_np = np.load(data)['features']
        
with open(cp_file, "rb") as data:
    cp_np = np.load(data)['features']
        
with open(mobc_file, "rb") as data:
    bc_mo_np = np.load(data)['features']

with open(similarity_fingerprints, "rb") as data:
    sim_fp = np.load(data)['features']

gemobc_np = np.concatenate((ge_np, bc_mo_np), axis=1)

with open('../misc/compounds16978to16170.npy', 'rb') as data:
    compounds_final_indicies_from_16978 = np.load(data)

mask=np.zeros(16978, dtype=bool)
mask[compounds_final_indicies_from_16978] = True
ge_np = ge_np[mask, :]
mo_np = mo_np[mask, :]
ge_scaled_np = ge_scaled_np[mask, :]
cp_np = cp_np[mask, :]
bc_mo_np = bc_mo_np[mask, :]
gemobc_np = gemobc_np[mask, :]
sim_fp = sim_fp[mask, :]

In [ ]:
#assay info is in csv files; we also store headers      
assay_all_df = pd.read_csv(assay_file)
assay_all_np = assay_all_df.to_numpy()
assay_header = assay_all_df.columns

In [ ]:
assay_all_np.shape

In [ ]:
# calculate sparsity of the assay matrix
for_sparsity = assay_all_np[:, 1:].astype(np.float64)
for_sparsity[for_sparsity == -1] = np.NaN
np.isnan(for_sparsity).sum()/np.prod(for_sparsity.shape)

In [ ]:
# calculate number of readouts in the assay matrix
np.count_nonzero(~np.isnan(for_sparsity)) 

In [ ]:
# calculate mean hit rate per assay
row_hits = np.sum(for_sparsity == 1,axis=1)
row_reads = np.sum(~np.isnan(for_sparsity),axis=1)
np.mean(row_hits/row_reads)

In [ ]:
# calculate readouts count per assay
reads = np.sum(~np.isnan(for_sparsity))
reads/270

In [ ]:
# calculate hits count per assay
hits = np.sum(for_sparsity == 1)
hits/270

In [ ]:
#readout percentiles
for_sparsity = assay_all_np[:, 1:].astype(np.float64)
for_sparsity[for_sparsity == -1] = np.NaN
b = np.sum(~np.isnan(for_sparsity),axis=0)
np.percentile(b, [5,25,50,75,95])


In [ ]:
# Function to create random splits (not cross validation)
def train_test_split(cp_np_array, gemobc_np_array, bc_mo_np_array, ge_np_array, mo_np_array, ge_scaled_np_array, assay_all_np_array ):
    indices_test = np.random.choice(cp_np_array.shape[0], int(cp_np_array.shape[0]*0.2), replace=False)
    mask=np.zeros(cp_np_array.shape[0], dtype=bool)
    mask[indices_test] = True
    return indices_test, bc_mo_np_array[~mask, :],  bc_mo_np_array[mask, :], cp_np_array[~mask, :], cp_np_array[mask, :], \
            gemobc_np_array[~mask, :], gemobc_np_array[mask, :], ge_np_array[~mask, :], ge_np_array[mask, :], mo_np_array[~mask, :], \
            mo_np_array[mask, :], ge_scaled_np_array[~mask, :], ge_scaled_np_array[mask, :], assay_all_np_array[~mask, :], assay_all_np_array[mask, :]

In [ ]:
def train_test_split_with_index_cv(indices_test, cp_np_array, gemobc_np_array, bc_mo_np_array, ge_np_array, mo_np_array, ge_scaled_np_array, assay_all_np_array ):
    mask=np.zeros(cp_np_array.shape[0], dtype=bool)
    mask[indices_test] = True
    return bc_mo_np_array[~mask, :],  bc_mo_np_array[mask, :], cp_np_array[~mask, :], cp_np_array[mask, :], \
            gemobc_np_array[~mask, :], gemobc_np_array[mask, :], ge_np_array[~mask, :], ge_np_array[mask, :], mo_np_array[~mask, :], \
            mo_np_array[mask, :], ge_scaled_np_array[~mask, :], ge_scaled_np_array[mask, :], assay_all_np_array[~mask, :], assay_all_np_array[mask, :]

In [ ]:
# create folders and files for random splits

for i in range(0,10):
    relative_path = '../data/random_{}'.format(i)
    os.mkdir(relative_path)
    os.chdir(relative_path)
    indices_test, bc_mo_np_train, bc_mo_np_test, cp_np_train, cp_np_test, gemobc_np_train, gemobc_np_test, ge_np_train, \
        ge_np_test, mo_np_train, mo_np_test, ge_scaled_np_train, ge_scaled_np_test, assay_all_np_train, assay_all_np_test = \
        train_test_split(cp_np, gemobc_np, bc_mo_np, ge_np, mo_np, ge_scaled_np, assay_all_np)
    
    print(ge_np_train.shape, mo_np_train.shape, assay_all_np_train.shape)
    
    assay_train_npdf = np.concatenate((np.array(assay_header)[np.newaxis, :], assay_all_np_train), axis=0)
    assay_test_npdf = np.concatenate((np.array(assay_header)[np.newaxis, :], assay_all_np_test), axis=0)
    
    assay_train_df = pd.DataFrame(data=assay_train_npdf[0:,0:])
    assay_test_df = pd.DataFrame(data=assay_test_npdf[0:,0:])
    
    assay_train_df.to_csv('assay_matrix_discrete_train_scaff.csv', header=False, index=False)
    assay_test_df.to_csv('assay_matrix_discrete_test_scaff.csv', header=False, index=False)
    
    #Commented out as those indices exist (can be found in the dataset files on Zenodo)
    #np.savez('random_split_indicies_' + str(i) + '.npz', features=indices_test)
    
    np.savez('population_gemobc_train.npz', features=gemobc_np_train)
    np.savez('population_gemobc_test.npz', features=gemobc_np_test)
    np.savez('population_ge_train.npz', features=ge_np_train)
    np.savez('population_ge_test.npz', features=ge_np_test)  
    np.savez('population_mo_train.npz', features=mo_np_train)
    np.savez('population_mo_test.npz', features=mo_np_test)
    np.savez('population_mobc_train.npz', features=bc_mo_np_train)
    np.savez('population_mobc_test.npz', features=bc_mo_np_test)  
    np.savez('population_ge_scaled_train.npz', features=ge_scaled_np_train)
    np.savez('population_ge_scaled_test.npz', features=ge_scaled_np_test)
    np.savez('population_cp_train.npz', features=cp_np_train)
    np.savez('population_cp_test.npz', features=cp_np_test)
    os.chdir('../../analysis/')
    

In [ ]:
# Generate compound order for cross-validation
all_indicies = np.arange(ge_np.shape[0])
np.random.shuffle(all_indicies)
print(all_indicies)

In [ ]:
# Save the array, commented out as it is already exists
#np.savez('./data/cross_validation_indicies_jan22.npz', features=all_indicies)

In [ ]:
# Create folders and files with cross-validation splits

with open('../splitting/cross_validation_indicies.npz', "rb") as data:
    all_indicies = np.load(data) 
    all_indicies = all_indicies['features']

for i in range(0,5):
    relative_path = '../data/CV_cv{}'.format(i)
    os.mkdir(relative_path)
    os.chdir(relative_path)
    if i < 4:
        bc_mo_np_train, bc_mo_np_test, cp_np_train, cp_np_test, gemobc_np_train, gemobc_np_test, ge_np_train, \
        ge_np_test, mo_np_train, mo_np_test, ge_scaled_np_train, ge_scaled_np_test, assay_all_np_train, assay_all_np_test = \
        train_test_split_with_index_cv(all_indicies[i*3234:i*3234+3234], cp_np, gemobc_np, bc_mo_np, ge_np, mo_np, ge_scaled_np, assay_all_np)
    else:
        bc_mo_np_train, bc_mo_np_test, cp_np_train, cp_np_test, gemobc_np_train, gemobc_np_test, ge_np_train, \
        ge_np_test, mo_np_train, mo_np_test, ge_scaled_np_train, ge_scaled_np_test, assay_all_np_train, assay_all_np_test = \
        train_test_split_with_index_cv(all_indicies[i*3234:], cp_np, gemobc_np, bc_mo_np, ge_np, mo_np, ge_scaled_np, assay_all_np)
    
    print(gemobc_np_train.shape, bc_mo_np_train.shape, ge_np_train.shape, mo_np_train.shape, assay_all_np_train.shape, np.array(assay_header)[:, np.newaxis].shape)
    print(gemobc_np_test.shape, bc_mo_np_test.shape, ge_np_test.shape, mo_np_test.shape, assay_all_np_test.shape, np.array(assay_header)[:, np.newaxis].shape)
    
    assay_train_npdf = np.concatenate((np.array(assay_header)[np.newaxis, :], assay_all_np_train), axis=0)
    assay_test_npdf = np.concatenate((np.array(assay_header)[np.newaxis, :], assay_all_np_test), axis=0)
    
    assay_train_df = pd.DataFrame(data=assay_train_npdf[0:,0:])
    assay_test_df = pd.DataFrame(data=assay_test_npdf[0:,0:])
    
    assay_train_df.to_csv('assay_matrix_discrete_train_scaff.csv', header=False, index=False)
    assay_test_df.to_csv('assay_matrix_discrete_test_scaff.csv', header=False, index=False)
    
    np.savez('population_gemobc_train.npz', features=gemobc_np_train)
    np.savez('population_gemobc_test.npz', features=gemobc_np_test)
    np.savez('population_ge_train.npz', features=ge_np_train)
    np.savez('population_ge_test.npz', features=ge_np_test)  
    np.savez('population_mo_train.npz', features=mo_np_train)
    np.savez('population_mo_test.npz', features=mo_np_test)
    np.savez('population_mobc_train.npz', features=bc_mo_np_train)
    np.savez('population_mobc_test.npz', features=bc_mo_np_test)  
    np.savez('population_ge_scaled_train.npz', features=ge_scaled_np_train)
    np.savez('population_ge_scaled_test.npz', features=ge_scaled_np_test)
    np.savez('population_cp_train.npz', features=cp_np_train)
    np.savez('population_cp_test.npz', features=cp_np_test)  
    os.chdir('../../analysis')

In [ ]:
# UMAP plot of morphology features
reducer = umap.UMAP()
embeddings = reducer.fit_transform(mo_np)
plt.figure(figsize=(10,10))
plt.scatter(x=embeddings[:,0], y=embeddings[:,1])

In [ ]:
# UMAP plot of batch-corrected morphology features
reducer = umap.UMAP()
embeddings = reducer.fit_transform(bc_mo_np)
plt.figure(figsize=(10,10))
plt.scatter(x=embeddings[:,0], y=embeddings[:,1])

In [ ]:
# UMAP plot of gene expression features
reducer = umap.UMAP()
embeddings = reducer.fit_transform(ge_np)
plt.figure(figsize=(10,10))
plt.scatter(x=embeddings[:,0], y=embeddings[:,1])

In [ ]:
# UMAP plot of classical chemical structures features
reducer = umap.UMAP()
embeddings = reducer.fit_transform(cp_np)
plt.figure(figsize=(10,10))
plt.scatter(x=embeddings[:,0], y=embeddings[:,1])

In [ ]:
# change the path here to the clustering file; this method is for morphology and gene expression clusters index files
def train_test_split_cluster(cp_np_array, gemobc_np_array, bc_mo_np_array, ge_np_array, mo_np_array, ge_scaled_np_array, assay_all_np_array, cluster):
    with open('../splitting/MOBC_clusters_size_constrained.npz', "rb") as data:
        clusters = np.load(data)
        indices_test = np.argwhere(clusters['features']==cluster)
        mask=np.zeros(cp_np_array.shape[0], dtype=bool)
        mask[indices_test] = True
        return bc_mo_np_array[~mask, :],  bc_mo_np_array[mask, :], cp_np_array[~mask, :], cp_np_array[mask, :], \
            gemobc_np_array[~mask, :], gemobc_np_array[mask, :], ge_np_array[~mask, :], ge_np_array[mask, :], mo_np_array[~mask, :], \
            mo_np_array[mask, :], ge_scaled_np_array[~mask, :], ge_scaled_np_array[mask, :], assay_all_np_array[~mask, :], assay_all_np_array[mask, :]

In [ ]:
# UMAP plots, then clustering of gene expression features, then makes an UMAP plots colored with clusters

reducer = umap.UMAP()
embeddings = reducer.fit_transform(ge_np)
plt.figure(figsize=(10,10))
plt.scatter(x=embeddings[:,0], y=embeddings[:,1])

model = equal.SameSizeKMeansHeuristics(5)
model.fit(ge_np)
y_kmeans = model.labels_

plt.figure(figsize=(10,10))
plt.scatter(embeddings[:, 0], embeddings[:, 1], c=y_kmeans, s=50, cmap='Pastel1')

In [ ]:
# save the array, commented out as it is already exists
#np.savez('geneexp_clusters_size_constrainedF.npz', features=y_kmeans)

In [ ]:
# UMAP plots, then clustering of batch-corrected morphology features, then makes an UMAP plots colored with clusters
reducer = umap.UMAP()
embeddings = reducer.fit_transform(bc_mo_np)
plt.figure(figsize=(10,10))
plt.scatter(x=embeddings[:,0], y=embeddings[:,1])

model = equal.SameSizeKMeansHeuristics(5)
model.fit(bc_mo_np)
y_kmeans = model.labels_

plt.figure(figsize=(10,10))
plt.scatter(embeddings[:, 0], embeddings[:, 1], c=y_kmeans, s=50, cmap='Pastel1')

In [ ]:
#np.savez('MOBC_clusters_size_constrained.npz', features=y_kmeans)

In [ ]:
# create folders and files for clustering based splits
for i in range(0,5):
    relative_path = '../data/MOBC_cv{}'.format(i)
    os.mkdir(relative_path)
    
    bc_mo_np_train, bc_mo_np_test, cp_np_train, cp_np_test, gemobc_np_train, gemobc_np_test, ge_np_train, \
        ge_np_test, mo_np_train, mo_np_test, ge_scaled_np_train, ge_scaled_np_test, assay_all_np_train, assay_all_np_test = \
        train_test_split_cluster(cp_np, gemobc_np, bc_mo_np, ge_np, mo_np, ge_scaled_np, assay_all_np, i)

    os.chdir(relative_path)
    
    print(bc_mo_np_train.shape, ge_np_train.shape, mo_np_train.shape, assay_all_np_train.shape, np.array(assay_header)[:, np.newaxis].shape)
    print(bc_mo_np_test.shape, ge_np_test.shape, mo_np_test.shape, assay_all_np_test.shape, np.array(assay_header)[:, np.newaxis].shape)
    
    assay_train_npdf = np.concatenate((np.array(assay_header)[np.newaxis, :], assay_all_np_train), axis=0)
    assay_test_npdf = np.concatenate((np.array(assay_header)[np.newaxis, :], assay_all_np_test), axis=0)
    
    assay_train_df = pd.DataFrame(data=assay_train_npdf[0:,0:])
    assay_test_df = pd.DataFrame(data=assay_test_npdf[0:,0:])
    
    assay_train_df.to_csv('assay_matrix_discrete_train_scaff.csv', header=False, index=False)
    assay_test_df.to_csv('assay_matrix_discrete_test_scaff.csv', header=False, index=False)
    
    np.savez('population_gemobc_train.npz', features=gemobc_np_train)
    np.savez('population_gemobc_test.npz', features=gemobc_np_test)
    np.savez('population_ge_train.npz', features=ge_np_train)
    np.savez('population_ge_test.npz', features=ge_np_test)  
    np.savez('population_mo_train.npz', features=mo_np_train)
    np.savez('population_mo_test.npz', features=mo_np_test)
    np.savez('population_mobc_train.npz', features=bc_mo_np_train)
    np.savez('population_mobc_test.npz', features=bc_mo_np_test)  
    np.savez('population_ge_scaled_train.npz', features=ge_scaled_np_train)
    np.savez('population_ge_scaled_test.npz', features=ge_scaled_np_test)
    np.savez('population_cp_train.npz', features=cp_np_train)
    np.savez('population_cp_test.npz', features=cp_np_test)  
    os.chdir('../../analysis')   

In [ ]:
with open('../splitting/scaffold_based_split.npz', 'rb') as data:
    chem_clusters = np.load(data, allow_pickle = True)
    chem_clusters = chem_clusters['features']
    
for i in range(0,5):
    relative_path = '../data/chemical_cv{}'.format(i)
    os.mkdir(relative_path)
    os.chdir(relative_path)

    assay_all_np[assay_all_np==-1]=np.nan
    bc_mo_np_train, bc_mo_np_test, cp_np_train, cp_np_test, gemobc_np_train, gemobc_np_test, ge_np_train, \
        ge_np_test, mo_np_train, mo_np_test, ge_scaled_np_train, ge_scaled_np_test, assay_all_np_train, assay_all_np_test = \
        train_test_split_with_index_cv(chem_clusters[i], cp_np, gemobc_np, bc_mo_np, ge_np, mo_np, ge_scaled_np, assay_all_np)
    
    print(gemobc_np_train.shape, bc_mo_np_train.shape, ge_np_train.shape, mo_np_train.shape, assay_all_np_train.shape, np.array(assay_header)[:, np.newaxis].shape)
    print(gemobc_np_test.shape, bc_mo_np_test.shape, ge_np_test.shape, mo_np_test.shape, assay_all_np_test.shape, np.array(assay_header)[:, np.newaxis].shape)
    
    assay_train_npdf = np.concatenate((np.array(assay_header)[np.newaxis, :], assay_all_np_train), axis=0)
    assay_test_npdf = np.concatenate((np.array(assay_header)[np.newaxis, :], assay_all_np_test), axis=0)
    
    assay_train_df = pd.DataFrame(data=assay_train_npdf[0:,0:])
    assay_test_df = pd.DataFrame(data=assay_test_npdf[0:,0:])
    
    
    assay_train_df.to_csv('assay_matrix_discrete_train_scaff.csv', header=False, index=False)
    assay_test_df.to_csv('assay_matrix_discrete_test_scaff.csv', header=False, index=False)
    
    np.savez('population_gemobc_train.npz', features=gemobc_np_train)
    np.savez('population_gemobc_test.npz', features=gemobc_np_test)
    np.savez('population_ge_train.npz', features=ge_np_train)
    np.savez('population_ge_test.npz', features=ge_np_test)  
    np.savez('population_mo_train.npz', features=mo_np_train)
    np.savez('population_mo_test.npz', features=mo_np_test)
    np.savez('population_mobc_train.npz', features=bc_mo_np_train)
    np.savez('population_mobc_test.npz', features=bc_mo_np_test)  
    np.savez('population_ge_scaled_train.npz', features=ge_scaled_np_train)
    np.savez('population_ge_scaled_test.npz', features=ge_scaled_np_test)
    np.savez('population_cp_train.npz', features=cp_np_train)
    np.savez('population_cp_test.npz', features=cp_np_test)  
    os.chdir('../../analysis')

In [ ]:
# plot UMAP of classical features + plot UMAP of classical features colored by scaffold-based splits
with open('../splitting/scaffold_based_split.npz', 'rb') as data:
    colors = np.zeros((cp_np.shape[0]))
    indices = np.load(data, allow_pickle=True)
    indices = indices['features']
    for i in range(5):
        colors[indices[i]] = i

reducer = umap.UMAP()
embeddings = reducer.fit_transform(cp_np)
plt.figure(figsize=(10,10))
plt.scatter(x=embeddings[:,0], y=embeddings[:,1])

plt.figure(figsize=(10,10))
plt.scatter(embeddings[:, 0], embeddings[:, 1], c=colors, s=50, cmap='Pastel1')